In [ ]:
%pip install numpy pandas pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 34.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import random
import pulp

np.random.seed(42)

In [ ]:
n_knapsacks = 50
n_items = 100
max_capacity = 100
min_capacity = 1
max_value = 10
max_weight = 100

def generate_knapsack_problem():
    knapsack_ids = [f"knapsack{i + 1}" for i in range(n_knapsacks)]
    capacities = np.random.randint(min_capacity, max_capacity, n_knapsacks)
    knapsack_df = pd.DataFrame({"id": knapsack_ids, "capacity": capacities })

    item_ids = [f"item{i + 1}" for i in range(n_items)]
    values = np.random.randint(1, max_value + 1, n_items)
    weights = np.random.randint(1, max_weight + 1, n_items)
    item_df = pd.DataFrame({"id": item_ids, "value": values, "weight": weights})

    return knapsack_df, item_df

In [ ]:
knapsack_df, item_df = generate_knapsack_problem()

In [ ]:
knapsack_df.head()

,id,capacity
0,knapsack1,52
1,knapsack2,93
2,knapsack3,15
3,knapsack4,72
4,knapsack5,61


In [ ]:
item_df.head()

,id,value,weight
0,item1,4,28
1,item2,9,28
2,item3,2,44
3,item4,10,84
4,item5,9,30


In [ ]:
def ip(values, weights, capacities):
  item_count = len(values)
  knapsack_count = len(capacities)

  problem = pulp.LpProblem("MultiKnapsack", pulp.LpMaximize)
  vars = [(i, k) for i in range(item_count) for k in range(knapsack_count)]
  decision_vars = pulp.LpVariable.dicts("item", vars, cat=pulp.LpBinary)

  problem += pulp.lpSum([values[i] * decision_vars[(i, k)] for i in range(item_count) for k in range(knapsack_count)])

  for i in range(item_count):
    problem  += pulp.lpSum([decision_vars[(i, k)] for k in range(knapsack_count)]) <= 1, f"OneKnapsack_{i}"

  for k in range(knapsack_count):
      problem += pulp.lpSum([weights[i] * decision_vars[(i, k)] for i in range(item_count)]) <= capacities[k], f"CapacityKnapsack_{k}"

  solver = pulp.PULP_CBC_CMD(timeLimit=600, msg=True)

  problem.solve(solver)

  is_optimal = pulp.LpStatus[problem.status] == 'Optimal'

  return {
      "status": pulp.LpStatus[problem.status],
      "total_value": pulp.value(problem.objective),
      "items_per_knapsack": {(i, k): pulp.value(decision_vars[(i, k)]) for i in range(item_count) for k in range(knapsack_count) if pulp.value(decision_vars[(i, k)]) == 1},
      "is_optimal": is_optimal
  }

In [ ]:
values = item_df['value'].values
weights = item_df['weight'].values
capacities = knapsack_df['capacity'].values

In [ ]:
result = ip(values, weights, capacities)
result_df = pd.DataFrame([result])
result_df

,status,total_value,items_per_knapsack,is_optimal
0,Optimal,456.0,"{(0, 4): 1.0, (1, 8): 1.0, (3, 16): 1.0, (4, 1...",True
